Chapter 9 stuff

In [ ]:
#!pip install strawberryfields

#CV quantum teleportation 

# This code is adopted from: https://strawberryfields.ai/photonics/demos/run_teleportation.html

#import modules 
import strawberryfields as sf
from strawberryfields.ops import *
import numpy as np
from numpy import pi, sqrt
from matplotlib import pyplot as plt

np.random.seed(42) #for reproducibility 

prog = sf.Program(3) #initializing the program using three quantum registers

alpha = 1+0.5j
r = np.abs(alpha)
phi = np.angle(alpha)

with prog.context as q:
    # prepare initial states
    Coherent(r, phi) | q[0]
    Squeezed(-2) | q[1]
    Squeezed(2) | q[2]

    # apply gates
    BS = BSgate(pi/4, pi)
    BS | (q[1], q[2])
    BS | (q[0], q[1])

    # Perform homodyne measurements
    MeasureX | q[0]
    MeasureP | q[1]

    # Displacement gates conditioned on
    # the measurements
    Xgate(sqrt(2) * q[0].par) | q[2]
    Zgate(sqrt(2) * q[1].par) | q[2]


engine = sf.Engine('fock', backend_options={"cutoff_dim": 15}) #select backend

result = engine.run(prog, shots=1, modes=None, compile_options={})

# Print results 
print(result.samples)
print(result.state)
state = result.state

print(state.dm().shape)

rho2 = np.einsum('kkllij->ij', state.dm())
print(rho2.shape)


probs = np.real_if_close(np.diagonal(rho2))
print(probs)


plt.bar(range(7), probs[:7])
plt.xlabel('Fock state')
plt.ylabel('Marginal probability')
plt.title('Mode 2')
plt.show()


fock_probs = state.all_fock_probs()
fock_probs.shape
np.sum(fock_probs, axis=(0,1))

In [ ]:
'''
This code is adopted from: www.pennylane.ai
import pennylane as qml
Continuous-variable qauntum variational algorithm.
'''

#!pip install pennylane --upgrade
#!pip install pennylane-qchem


import pennylane as qml
from pennylane import qchem 
from pennylane import numpy as np #use numpy from pennylane instead of the standard numpy
#from matplotlib import pyplot as plt

np.random.seed(42)

geometry = 'h2.xyz'
name ='h2'
charge = 0
multiplicity=1
basis= 'sto-3g' 
#h, nr_qubits = qml.qchem.generate_hamiltonian(
h, nr_qubits = qchem.generate_hamiltonian(
    name,
    geometry,
    charge,
    multiplicity,
    basis,
    mapping='jordan_wigner',
    n_active_orbitals=2,
    n_active_electrons=2,
)

print("Hamiltonian is: \n", h)
print("Number of qubits is: \n", nr_qubits)

geometry = 'h2.xyz'
name ='h2'
charge = 0
multiplicity=1
basis= 'sto-3g' 
#h, nr_qubits = qml.qchem.generate_hamiltonian(
h, nr_qubits = qml.qchem.molecular_hamiltonian(
    name,
    geometry,
    charge,
    multiplicity,
    basis,
    mapping='jordan_wigner',
    n_active_orbitals=2,
    n_active_electrons=2,
)

print("Hamiltonian is: \n", h)
print("Number of qubits is: \n", nr_qubits)

opt = qml.GradientDescentOptimizer(stepsize=0.4)
params = np.random.normal(0, np.pi, (nr_qubits, 3))

print(params)


max_iterations = 250
step_size = 0.05
conv_tol = 1e-06

prev_energy = cost_fn(params)
for n in range(max_iterations):
    params = opt.step(cost_fn, params)
    energy = cost_fn(params)
    conv = np.abs(energy - prev_energy)

print()
print('Final convergence parameter = {:.8f} Ha'.format(conv))
print('Final value of the ground-state energy = {:.8f} Ha'.format(energy))
print('Accuracy with respect to the FCI energy: {:.8f} Ha ({:.8f} kcal/mol)'.
        format(np.abs(energy - (-1.136189454088)), np.abs(energy - (-1.136189454088))*627.503))
print()
print('Final circuit parameters = \n', params)